In [ ]:
# Install selenium, beutifulsoup.
# Update google chrome and install chromedriver for the updated version.
# Install chromedriver inside /usr/bin/

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.parse import urlparse
import time
import pickle

In [ ]:
#This cell needs to be executed in one go and it will take quite some time depending on number of ads.
#Once pickle file is formed correctly no need to execute it anymore.


WAIT_TIME = 1.0

# no_of_clicks define how many times "LOAD MORE" button has to be clicked on the page to load all the ads.
# For each click, 20 new ads are loaded.
# Eg. if total 22663 ads are there then set no_of_clicks = ceil(22663/20) + (some 10 or 20 extra clicks for safety).
no_of_clicks = 1160

def loadAllAds(driver):
    
    for i in range(no_of_clicks):
        try:
            time.sleep(WAIT_TIME)
            xpath = "//button[@data-aut-id='btnLoadMore']"
            load_more = driver.find_elements_by_xpath(xpath)
            load_more[0].click()
            time.sleep(WAIT_TIME)
            print("done",i)
        except:
            print("error",i)
            pass

# path to the chromedriver
driver = webdriver.Chrome("/usr/bin/chromedriver")
#change this url according to your category
driver.get("https://www.olx.in/en/delhi_g4058659/for-sale-houses-apartments_c1725?sorting=asc-price&filter=listed_by_eq_broker")
loadAllAds(driver)
content = driver.page_source

web_pickle_file = open('webpage','wb')
pickle.dump(content,web_pickle_file)
web_pickle_file.close()

In [ ]:
#This cell can executed in phased manner by changing START & STOP values.

web_pickle_file = open('webpage','rb')
content = pickle.load(web_pickle_file)
soup = BeautifulSoup(content)

def getLatLong(query):
    lat=""
    long=""
    i=7
    while query[i]!='%':
        lat += query[i]
        i+=1
        
    i+=3
    while query[i]!='&':
        long += query[i]
        i+=1        
    return lat,long


Latitude=[]
Longitude=[]
Address=[]
Type=[]
UnprocessedAds=[]

ad_list=soup.find('ul',{"class":"rl3f9 _3mXOU"})
ads=ad_list.find_all("li",{"data-aut-id":"itemBox"})
print("Total ads:",len(ads))

# len(ads) tells total number of ads that needs to be scraped.
# START & STOP values range between 0 and len(ads).
# To scrape all ads in one go, set START=0 & STOP=len(ads)
START = 0
STOP = len(ads)

# prints log after every LogAfter ads are scraped.
LogAfter = 100

for i in range(START,STOP):
    try:
        link = ads[i].find_all('a')
        driver.get("https://www.olx.in"+link[0]["href"])
        ad_soup = BeautifulSoup(driver.page_source)
        div = ad_soup.findAll("div", {"class": "_1ohyB YYU0x"})
        image = div[0].findAll("img");
        url = image[0]["src"]
        parsed = urlparse(url)
        lat,long = getLatLong(parsed.query)
        span = ad_soup.findAll("span",{"class":"_2FRXm"})
        addr = span[0].text
        #Set type name according to your category.
        Type.append("Houses & Apartments")
        Latitude.append(lat)
        Longitude.append(long)
        Address.append(addr)
        
        if i%LogAfter == 0:
            print("Scraped",i,"ads till now.")
    except:
        UnprocessedAds.append(i)
        pass


df = pd.DataFrame({'Type':Type,'Address':Address,'Latitude':Latitude,'Longitude':Longitude}) 
#If you are scraping in phased manner then each time give a new name to file.
df.to_csv('olx_ads.csv', index=False, encoding='utf-8')
print("File stored successfully.")

print("Total unprocessed ads:",len(UnprocessedAds))
# if total unprocessed ads are less then there is no problem. But if they are lot more then we need to
# process those indices which are stored in UnprocessedAds again.